# Build and test the inference container

This notebook is linked to https://eoap.github.io/inference-eoap/containers/inference/

## Goal

Create a container and run the `inference` step in the container image.

## Dependencies

This notebook uses the output of the execution of the steps described in https://eoap.github.io/inference-eoap/cwl-workflow/stage-and-cog/

## Kernel

This notebook uses the `bash` kernel

## Setup the environment

In [ ]:
export WORKSPACE=/workspace/inference-eoap
export RUNTIME=${WORKSPACE}/runs

mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [ ]:
cat ${WORKSPACE}/tile-based-classification/command-line-tools/inference/Dockerfile

Build the container using `podman`:

In [ ]:
podman build --format docker -t localhost/inference:latest ${WORKSPACE}/tile-based-classification/command-line-tools/inference


Show the `inference` help:

In [ ]:
podman run --rm -it --env=PYTHONPATH=/app localhost/inference:latest python -m app --help

## Test the inference step in the container

In [ ]:
# check if the results.json file exists
if [ ! -f ${WORKSPACE}/runs/results.json ]; then
    echo "results.json file not found, run the stage-and-cog.cwl workflow first."
    exit 1
fi

podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/inference-eoap/runs,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/inference:latest \
    python \
    -m \
    app \
    --input-item \
    $( cat ${WORKSPACE}/runs/results.json | jq -r .stac_catalog.path | sed 's/\/workspace\/inference-eoap//' )

List the outputs:

In [ ]:
tree ${RUNTIME}